In [212]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch, onecold, crossentropy, throttle
using Flux: @epochs
using Statistics
using MLDatasets: SVHN2
using Serialization: serialize, deserialize
using PyCall
using ImageView
using Images
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra
using Metalhead: trainimgs

In [213]:
x_train, y_train = SVHN2.traindata()
x_test, y_test = SVHN2.testdata();

In [214]:
epochs = 100;
batch_size = 512;

In [215]:
train_imgs = [Float32.(x_train[:, :, :, i]) for i in 1:size(x_train)[4]]
train_labels = onehotbatch(reshape(y_train, (1, size(y_train)[1])), 1:10)
train_X = [(cat(train_imgs[i]..., dims = 4), train_labels[:, i]) for i in partition(1:size(x_train)[4], batch_size)];

In [216]:
test_imgs = [Float32.(x_test[:, :, :, i]) for i in 1:5000]
test_labels = onehotbatch(y_test[1:5000], 1:10)
test_X = cat(test_imgs..., dims = 4)
test_Y = [test_labels[:, i] for i in 1:5000];

In [238]:
model = Chain(
    Conv((3, 3), 3=>8, pad=(1, 1), stride=(1, 1), relu),
#     BatchNorm(8),
#     Conv((3, 3), 8=>16, pad=(1, 1), stride=(1, 1), relu),
#     BatchNorm(16),
    MeanPool((4, 4)),
    x -> reshape(x, :, size(x, 4)),
#     Dense(1024, 64, relu),
    Dense(512, 10),
    softmax
);

# vgg16() = Chain(
#   Conv((3, 3), 3 => 64, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(64),
#   Conv((3, 3), 64 => 64, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(64),
#   x -> maxpool(x, (2, 2)),
#   Conv((3, 3), 64 => 128, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(128),
#   Conv((3, 3), 128 => 128, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(128),
#   x -> maxpool(x, (2,2)),
#   Conv((3, 3), 128 => 256, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(256),
#   Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(256),
#   Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(256),
#   x -> maxpool(x, (2, 2)),
#   Conv((3, 3), 256 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   x -> maxpool(x, (2, 2)),
#   Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
#   BatchNorm(512),
#   x -> maxpool(x, (2, 2)),
#   x -> reshape(x, :, size(x, 4)),
#   Dense(512, 4096, relu),
#   Dropout(0.5),
#   Dense(4096, 4096, relu),
#   Dropout(0.5),
#   Dense(4096, 10),
#   softmax) |> gpu


# model = vgg16();

In [239]:
# function loss(x, y)
#     x_aug = x .+ 0.1f0*randn(eltype(x), size(x)) #szum losowy
#     y_hat = model(x_aug)
#     return crossentropy(y_hat, y)
# end
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(model(x), 1:10) .== onecold(y, 1:10))
opt = ADAM(0.1)

ADAM(0.1, (0.9, 0.999), IdDict{Any,Any}())

In [240]:
model(train_X[1][1])

10×512 Array{Float32,2}:
 0.0959018  0.101608   0.0866308  …  0.100368   0.0886092  0.101028
 0.107984   0.101576   0.0773607     0.0798235  0.0777328  0.0853259
 0.0829411  0.0873018  0.0510149     0.0627957  0.0687249  0.078302
 0.115831   0.120153   0.195496      0.148684   0.139935   0.135275
 0.102123   0.0991903  0.0988453     0.128653   0.138884   0.100421
 0.0857243  0.0853321  0.0609623  …  0.069858   0.0693432  0.085004
 0.101604   0.0969311  0.0991502     0.0764188  0.0978533  0.0987331
 0.11636    0.121434   0.187132      0.164439   0.165542   0.1321
 0.100086   0.0975262  0.0880451     0.108173   0.0961085  0.104078
 0.0914444  0.088948   0.0553633     0.0607863  0.0572679  0.0797329

In [241]:
accuracy(test_X, test_labels)

0.0928

In [244]:
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch = 1:epochs
    global best_acc, last_improvement
    Flux.train!(loss, params(model), train_X, opt)
    acc = accuracy(test_X, test_labels)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to SVHN_conv.bson")
        BSON.@save "SVHN_conv.bson" model epoch acc
        best_acc = acc
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[244]:1
┌ Info: [1]: Test accuracy: 0.3652
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [2]: Test accuracy: 0.4586
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [3]: Test accuracy: 0.5176
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [4]: Test accuracy: 0.5638
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [5]: Test accuracy: 0.5984
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [6]: Test accuracy: 0.6212
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [7]: Test accuracy: 0.6398
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to S

┌ Info: [61]: Test accuracy: 0.7680
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [62]: Test accuracy: 0.7686
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [63]: Test accuracy: 0.7698
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [64]: Test accuracy: 0.7698
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [65]: Test accuracy: 0.7708
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [66]: Test accuracy: 0.7718
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [67]: Test accuracy: 0.7718
└ @ Main In[244]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_conv.bson
└ @ Main In[244]:10
┌ Info: [68]: